In [1]:
import sys
import os
import pandas as pd

sys.path.append(os.path.abspath("../../ml/"))

from dev_wtp_io_utils import *

/home/bsmith16/.conda/envs/neuralsignature/lib/python3.8/site-packages/nilearn/datasets/__init__.py:93: FutureWarning: Fetchers from the nilearn.datasets module will be updated in version 0.9 to return python strings instead of bytes and Pandas dataframes instead of Numpy arrays.
  warn("Fetchers from the nilearn.datasets module will be "


In [2]:
# nonbids_data_path = "/gpfs/projects/sanlab/shared/DEV/nonbids_data/"
# ml_data_folderpath = "/gpfs/projects/sanlab/shared/DEV/nonbids_data/fMRI/ml"

sys.path.append(os.path.abspath("../../ml/"))
from apply_loocv_and_save import load_and_preprocess
from dev_utils import read_yaml_for_host, filter_dev_data
import warnings


config_data = read_yaml_for_host("sst_config.yml")



python initialized for apply_loocv_and_save
cpus available; cpus to use:
28 25
28
yaml file does not contain host-specific settings for this host. Using default settings.


In [3]:
nonbids_data_path = config_data['nonbids_data_path']
ml_data_folderpath = nonbids_data_path + "fMRI/ml"
dropbox_datapath = config_data['dropbox_datapath']


In [4]:
#test_train_df_raw = pd.read_csv(nonbids_data_path + "fMRI/ml/train_test_markers_20210601T183243.csv")
#test_train_df_raw = pd.read_csv(nonbids_data_path + "fMRI/ml/train_test_markers_20211027T173724.csv")
test_train_df_raw = pd.read_csv(nonbids_data_path + "fMRI/ml/train_test_markers_20230102T164214.csv")
train_subjs = filter_dev_data(test_train_df_raw, nonbids_data_path,dropbox_datapath)


In [5]:
def get_Brain_Data_for_sub(sub_label,behavdesign,betaseries_path='/gpfs/projects/sanlab/shared/DEV/nonbids_data/fMRI/fx/models/SST/wave1/betaseries_nos/'):
    return(dev_wtp_io_utils.get_Brain_Data_betas_for_sub(sub_label,behavdesign,betaseries_path))

In [6]:
from dev_wtp_io_utils import import_sst_cond_w1_subjs_to_pkl

In [7]:
all_sst_events= pd.read_csv(ml_data_folderpath +"/SST/" + "all_sst_events.csv")
#cut out other waves
sst_1_behavdesign_1 = all_sst_events[(all_sst_events.wave=='wave1')].reset_index(drop=True)
#label the ITI as such
sst_1_behavdesign_1.loc[(all_sst_events.trial_type.isna()),'trial_type']='ITI'
#ensure it is correctly ordered
sst_1_behavdesign_clean = sst_1_behavdesign_1.sort_values(['subject','onset']).reset_index(drop=True)
#now tell it the name of the corresponding beta. this is easy; they are all consecutive
sst_1_behavdesign_clean['beta'] = 'beta_' + (sst_1_behavdesign_clean.groupby(['subject','wave']).cumcount()+1).astype(str).apply(lambda x: x.zfill(4)) + '.nii'

In [8]:
all_sst_events[(all_sst_events.subject=='DEV013') & (all_sst_events.wave=="wave1")]

,onset,duration,trial_type,subject,wave
79872,0.00000,2.25834,correct-stop,DEV013,wave1
79873,2.25834,0.50000,NaN,DEV013,wave1
79874,2.75834,2.00556,failed-go,DEV013,wave1
79875,4.76390,0.75000,NaN,DEV013,wave1
79876,5.51390,0.48810,correct-go,DEV013,wave1
...,...,...,...,...,...
80123,412.82572,1.75000,NaN,DEV013,wave1
80124,414.57572,0.56873,correct-go,DEV013,wave1
80125,417.08682,0.75000,NaN,DEV013,wave1
80126,417.83682,1.50000,failed-stop,DEV013,wave1


In [9]:
#I want to import this other module; we don't actually use it but it's a good reference
#for future programmers to see
#it's important we keep the logic consistent for the two main conditions
sys.path.append(os.path.abspath("../../fx/multiconds/SST/"))
from multiconds import create_posterror_masks_from_masks

#sst_1_behavdesign_clean['prev_trial_type'] = sst_1_behavdesign_clean['trial_type']
sst_1_behavdesign_clean['prev_trial_type'] = (
    sst_1_behavdesign_clean
    .loc[sst_1_behavdesign_clean['trial_type']!='ITI']
    .groupby(['subject','wave'])
    .shift(1)
    ['trial_type']
)#this works because during assignment, items will be assigned to the line corresponding to their index
#very important you don't reset index during this operation.
is_post_stop_correct_go = (
    (sst_1_behavdesign_clean['trial_type']=='correct-go') & 
    (sst_1_behavdesign_clean['prev_trial_type'].isin(['failed-stop','correct-stop']))
)

sst_1_behavdesign_clean.loc[is_post_stop_correct_go,'post_stop_correct_go_subtype'] = 'Other'
sst_1_behavdesign_clean.loc[is_post_stop_correct_go & (sst_1_behavdesign_clean['prev_trial_type']=='failed-stop'),'post_stop_correct_go_subtype'] = 'post-error'
sst_1_behavdesign_clean.loc[is_post_stop_correct_go & (sst_1_behavdesign_clean['prev_trial_type']=='correct-stop'),'post_stop_correct_go_subtype'] = 'post-correct'


In [10]:
sst_1_behavdesign_clean.groupby(['trial_type','post_stop_correct_go_subtype']).size().reset_index()

,trial_type,post_stop_correct_go_subtype,0
0,correct-go,post-correct,1879
1,correct-go,post-error,2145


In [11]:
sst_1_behavdesign_clean['trial_type'].value_counts()

ITI             26112
correct-go      18750
failed-stop      3434
correct-stop     3094
failed-go         834
Name: trial_type, dtype: int64

In [12]:
behavdesign = sst_1_behavdesign_clean
sst_1_behavdesign_clean_dev005 = behavdesign[[s in 'DEV005' for s in behavdesign.subject]]

## Get supplementary data

In [13]:
individual_differences = pd.read_csv(ml_data_folderpath + "/data_by_ppt.csv")
individual_differences = individual_differences.rename(columns={'SID':'subject'})
individual_differences['wave']=1
#individual_differences['wave'] = individual_differences['wave'].astype(object) # for compatibility with the wave column in the dataset

## Just subjects in the conditions of interest: CorrectStop vs. CorrectGo

In [14]:
sst_1_behavdesign_clean.trial_type.value_counts()

ITI             26112
correct-go      18750
failed-stop      3434
correct-stop     3094
failed-go         834
Name: trial_type, dtype: int64

In [15]:
sst_1_behavdesign_correct_conditions=sst_1_behavdesign_clean[sst_1_behavdesign_clean.trial_type.isin(['correct-go','correct-stop'])]

In [16]:
sst_1_behavdesign_correct_conditions.trial_type.value_counts()

correct-go      18750
correct-stop     3094
Name: trial_type, dtype: int64

### Now mask those subjects

In [17]:
# get the PFC mask
mask_pfc = nib.load(ml_data_folderpath + '/prefrontal_cortex.nii.gz')

In [18]:
train_subjs[0:2]

['DEV020', 'DEV047']

In [19]:
import_sst_cond_w1_subjs_to_pkl(
    train_subjs[0:2],'conditions_nos',
    out_folder = ml_data_folderpath + "/SST/",
    conditions_to_include = ['CorrectGo','CorrectStop'],
    condition_count_required=2,
    out_file_suffix = '_correct_cond_pfc',
    sst_wt_repo = nonbids_data_path + "fMRI/fx/models/SST/wave1/",
    subj_brain_data_args = {
        'standardize_by_condition':False,
        'mask':mask_pfc
    }
)

/gpfs/projects/sanlab/shared/DEV/nonbids_data/fMRI/fx/models/SST/wave1/conditions_nos/
DEV020
   condition_index condition_label  \
0                0       CorrectGo   
1                1     CorrectStop   

                            raw_beta_description           beta subject  wave  
0    spm_spm:beta (0001) - Sn(1) CorrectGo*bf(1)  beta_0001.nii  DEV020     1  
1  spm_spm:beta (0002) - Sn(1) CorrectStop*bf(1)  beta_0002.nii  DEV020     1  
.....verified that all expected betas exist! Processing...retrieving a mask from the first image in the series beta_0001.nii
...using a combined mask of the first image and the passed in masktemp mask created.
loading files. This step may take some time...
converting to nifti and fdata
   condition_index condition_label  \
0                0       CorrectGo   
1                1     CorrectStop   

                            raw_beta_description           beta subject  wave  
0    spm_spm:beta (0001) - Sn(1) CorrectGo*bf(1)  beta_0001.nii  DEV0

In [20]:
import_sst_cond_w1_subjs_to_pkl(
    train_subjs[0:6],'conditions_nos',
    out_folder = ml_data_folderpath + "/SST/",
    conditions_to_include = ['CorrectGo','CorrectStop'],
    condition_count_required=2,
    out_file_suffix = '_correct_cond_pfc',
    sst_wt_repo = nonbids_data_path + "fMRI/fx/models/SST/wave1/",
    subj_brain_data_args = {
        'standardize_by_condition':False,
        'mask':mask_pfc
    }
)

/gpfs/projects/sanlab/shared/DEV/nonbids_data/fMRI/fx/models/SST/wave1/conditions_nos/
DEV020
   condition_index condition_label  \
0                0       CorrectGo   
1                1     CorrectStop   

                            raw_beta_description           beta subject  wave  
0    spm_spm:beta (0001) - Sn(1) CorrectGo*bf(1)  beta_0001.nii  DEV020     1  
1  spm_spm:beta (0002) - Sn(1) CorrectStop*bf(1)  beta_0002.nii  DEV020     1  
.....verified that all expected betas exist! Processing...retrieving a mask from the first image in the series beta_0001.nii
...using a combined mask of the first image and the passed in masktemp mask created.
loading files. This step may take some time...
converting to nifti and fdata
   condition_index condition_label  \
0                0       CorrectGo   
1                1     CorrectStop   

                            raw_beta_description           beta subject  wave  
0    spm_spm:beta (0001) - Sn(1) CorrectGo*bf(1)  beta_0001.nii  DEV0

In [22]:
import_sst_cond_w1_subjs_to_pkl(
    train_subjs[0:30],'conditions_nos',
    out_folder = ml_data_folderpath + "/SST/",
    conditions_to_include = ['CorrectGo','CorrectStop'],
    condition_count_required=2,
    out_file_suffix = '_correct_cond_pfc',
    sst_wt_repo = nonbids_data_path + "fMRI/fx/models/SST/wave1/",
    subj_brain_data_args = {
        'standardize_by_condition':False,
        'mask':mask_pfc
    }
)

/gpfs/projects/sanlab/shared/DEV/nonbids_data/fMRI/fx/models/SST/wave1/conditions_nos/
DEV020
   condition_index condition_label  \
0                0       CorrectGo   
1                1     CorrectStop   

                            raw_beta_description           beta subject  wave  
0    spm_spm:beta (0001) - Sn(1) CorrectGo*bf(1)  beta_0001.nii  DEV020     1  
1  spm_spm:beta (0002) - Sn(1) CorrectStop*bf(1)  beta_0002.nii  DEV020     1  
.....verified that all expected betas exist! Processing...retrieving a mask from the first image in the series beta_0001.nii
...using a combined mask of the first image and the passed in masktemp mask created.
loading files. This step may take some time...
converting to nifti and fdata
   condition_index condition_label  \
0                0       CorrectGo   
1                1     CorrectStop   

                            raw_beta_description           beta subject  wave  
0    spm_spm:beta (0001) - Sn(1) CorrectGo*bf(1)  beta_0001.nii  DEV0

   condition_index condition_label  \
0                0       CorrectGo   
1                1     CorrectStop   

                            raw_beta_description           beta subject  wave  
0    spm_spm:beta (0001) - Sn(1) CorrectGo*bf(1)  beta_0001.nii  DEV100     1  
1  spm_spm:beta (0002) - Sn(1) CorrectStop*bf(1)  beta_0002.nii  DEV100     1  
.....verified that all expected betas exist! Processing...retrieving a mask from the first image in the series beta_0001.nii
...using a combined mask of the first image and the passed in masktemp mask created.
loading files. This step may take some time...
converting to nifti and fdata
   condition_index condition_label  \
0                0       CorrectGo   
1                1     CorrectStop   

                            raw_beta_description           beta subject  wave  
0    spm_spm:beta (0001) - Sn(1) CorrectGo*bf(1)  beta_0001.nii  DEV100     1  
1  spm_spm:beta (0002) - Sn(1) CorrectStop*bf(1)  beta_0002.nii  DEV100     1  
...

.....verified that all expected betas exist! Processing...retrieving a mask from the first image in the series beta_0001.nii
...using a combined mask of the first image and the passed in masktemp mask created.
loading files. This step may take some time...
converting to nifti and fdata
   condition_index condition_label  \
0                0       CorrectGo   
1                1     CorrectStop   

                            raw_beta_description           beta subject  wave  
0    spm_spm:beta (0001) - Sn(1) CorrectGo*bf(1)  beta_0001.nii  DEV118     1  
1  spm_spm:beta (0002) - Sn(1) CorrectStop*bf(1)  beta_0002.nii  DEV118     1  
...imported data.
DEV119
   condition_index condition_label  \
0                0       CorrectGo   
1                1     CorrectStop   

                            raw_beta_description           beta subject  wave  
0    spm_spm:beta (0001) - Sn(1) CorrectGo*bf(1)  beta_0001.nii  DEV119     1  
1  spm_spm:beta (0002) - Sn(1) CorrectStop*bf(1)  beta_000

.....verified that all expected betas exist! Processing...retrieving a mask from the first image in the series beta_0001.nii
...using a combined mask of the first image and the passed in masktemp mask created.
loading files. This step may take some time...
converting to nifti and fdata
   condition_index condition_label  \
0                0       CorrectGo   
1                1     CorrectStop   

                            raw_beta_description           beta subject  wave  
0    spm_spm:beta (0001) - Sn(1) CorrectGo*bf(1)  beta_0001.nii  DEV157     1  
1  spm_spm:beta (0002) - Sn(1) CorrectStop*bf(1)  beta_0002.nii  DEV157     1  
...imported data.
DEV158
   condition_index condition_label  \
0                0       CorrectGo   
1                1     CorrectStop   

                            raw_beta_description           beta subject  wave  
0    spm_spm:beta (0001) - Sn(1) CorrectGo*bf(1)  beta_0001.nii  DEV158     1  
1  spm_spm:beta (0002) - Sn(1) CorrectStop*bf(1)  beta_000

In [19]:
import_sst_cond_w1_subjs_to_pkl(
    train_subjs,'conditions_nos',
    out_folder = ml_data_folderpath + "/SST/",
    conditions_to_include = ['CorrectGo','CorrectStop'],
    condition_count_required=2,
    out_file_suffix = '_correct_cond_pfc',
    sst_wt_repo = nonbids_data_path + "fMRI/fx/models/SST/wave1/",
    subj_brain_data_args = {
        'standardize_by_condition':False,
        'mask':mask_pfc
    }
)

/gpfs/projects/sanlab/shared/DEV/nonbids_data/fMRI/fx/models/SST/wave1/conditions_nos/
DEV006
   condition_index condition_label  \
0                0       CorrectGo   
1                1     CorrectStop   

                            raw_beta_description           beta subject  wave  
0    spm_spm:beta (0001) - Sn(1) CorrectGo*bf(1)  beta_0001.nii  DEV006     1  
1  spm_spm:beta (0002) - Sn(1) CorrectStop*bf(1)  beta_0002.nii  DEV006     1  
.....verified that all expected betas exist! Processing...retrieving a mask from the first image in the series beta_0001.nii
...using a combined mask of the first image and the passed in masktemp mask created.
loading files. This step may take some time...
converting to nifti and fdata
   condition_index condition_label  \
0                0       CorrectGo   
1                1     CorrectStop   

                            raw_beta_description           beta subject  wave  
0    spm_spm:beta (0001) - Sn(1) CorrectGo*bf(1)  beta_0001.nii  DEV0

   condition_index condition_label  \
0                0       CorrectGo   
1                1     CorrectStop   

                            raw_beta_description           beta subject  wave  
0    spm_spm:beta (0001) - Sn(1) CorrectGo*bf(1)  beta_0001.nii  DEV021     1  
1  spm_spm:beta (0002) - Sn(1) CorrectStop*bf(1)  beta_0002.nii  DEV021     1  
.....verified that all expected betas exist! Processing...retrieving a mask from the first image in the series beta_0001.nii
...using a combined mask of the first image and the passed in masktemp mask created.
loading files. This step may take some time...
converting to nifti and fdata
   condition_index condition_label  \
0                0       CorrectGo   
1                1     CorrectStop   

                            raw_beta_description           beta subject  wave  
0    spm_spm:beta (0001) - Sn(1) CorrectGo*bf(1)  beta_0001.nii  DEV021     1  
1  spm_spm:beta (0002) - Sn(1) CorrectStop*bf(1)  beta_0002.nii  DEV021     1  
...

   condition_index condition_label  \
0                0       CorrectGo   
1                1     CorrectStop   

                            raw_beta_description           beta subject  wave  
0    spm_spm:beta (0001) - Sn(1) CorrectGo*bf(1)  beta_0001.nii  DEV030     1  
1  spm_spm:beta (0002) - Sn(1) CorrectStop*bf(1)  beta_0002.nii  DEV030     1  
.....verified that all expected betas exist! Processing...retrieving a mask from the first image in the series beta_0001.nii
...using a combined mask of the first image and the passed in masktemp mask created.
loading files. This step may take some time...
converting to nifti and fdata
   condition_index condition_label  \
0                0       CorrectGo   
1                1     CorrectStop   

                            raw_beta_description           beta subject  wave  
0    spm_spm:beta (0001) - Sn(1) CorrectGo*bf(1)  beta_0001.nii  DEV030     1  
1  spm_spm:beta (0002) - Sn(1) CorrectStop*bf(1)  beta_0002.nii  DEV030     1  
...

.....verified that all expected betas exist! Processing...retrieving a mask from the first image in the series beta_0001.nii
...using a combined mask of the first image and the passed in masktemp mask created.
loading files. This step may take some time...
converting to nifti and fdata
   condition_index condition_label  \
0                0       CorrectGo   
1                1     CorrectStop   

                            raw_beta_description           beta subject  wave  
0    spm_spm:beta (0001) - Sn(1) CorrectGo*bf(1)  beta_0001.nii  DEV042     1  
1  spm_spm:beta (0002) - Sn(1) CorrectStop*bf(1)  beta_0002.nii  DEV042     1  
...imported data.
DEV043
   condition_index condition_label  \
0                0       CorrectGo   
1                1     CorrectStop   

                            raw_beta_description           beta subject  wave  
0    spm_spm:beta (0001) - Sn(1) CorrectGo*bf(1)  beta_0001.nii  DEV043     1  
1  spm_spm:beta (0002) - Sn(1) CorrectStop*bf(1)  beta_000

.....verified that all expected betas exist! Processing...retrieving a mask from the first image in the series beta_0001.nii
...using a combined mask of the first image and the passed in masktemp mask created.
loading files. This step may take some time...
converting to nifti and fdata
   condition_index condition_label  \
0                0       CorrectGo   
1                1     CorrectStop   

                            raw_beta_description           beta subject  wave  
0    spm_spm:beta (0001) - Sn(1) CorrectGo*bf(1)  beta_0001.nii  DEV051     1  
1  spm_spm:beta (0002) - Sn(1) CorrectStop*bf(1)  beta_0002.nii  DEV051     1  
...imported data.
DEV052
   condition_index condition_label  \
0                0       CorrectGo   
1                1     CorrectStop   

                            raw_beta_description           beta subject  wave  
0    spm_spm:beta (0001) - Sn(1) CorrectGo*bf(1)  beta_0001.nii  DEV052     1  
1  spm_spm:beta (0002) - Sn(1) CorrectStop*bf(1)  beta_000

...

.....verified that all expected betas exist! Processing...retrieving a mask from the first image in the series beta_0001.nii
...using a combined mask of the first image and the passed in masktemp mask created.
loading files. This step may take some time...
converting to nifti and fdata
   condition_index condition_label  \
0                0       CorrectGo   
1                1     CorrectStop   

                            raw_beta_description           beta subject  wave  
0    spm_spm:beta (0001) - Sn(1) CorrectGo*bf(1)  beta_0001.nii  DEV252     1  
1  spm_spm:beta (0002) - Sn(1) CorrectStop*bf(1)  beta_0002.nii  DEV252     1  
...imported data.
DEV254
   condition_index condition_label  \
0                0       CorrectGo   
1                1     CorrectStop   

                            raw_beta_description           beta subject  wave  
0    spm_spm:beta (0001) - Sn(1) CorrectGo*bf(1)  beta_0001.nii  DEV254     1  
1  spm_spm:beta (0002) - Sn(1) CorrectStop*bf(1)  beta_000

.....verified that all expected betas exist! Processing...retrieving a mask from the first image in the series beta_0001.nii
...using a combined mask of the first image and the passed in masktemp mask created.
loading files. This step may take some time...
converting to nifti and fdata
   condition_index condition_label  \
0                0       CorrectGo   
1                1     CorrectStop   

                            raw_beta_description           beta subject  wave  
0    spm_spm:beta (0001) - Sn(1) CorrectGo*bf(1)  beta_0001.nii  DEV265     1  
1  spm_spm:beta (0002) - Sn(1) CorrectStop*bf(1)  beta_0002.nii  DEV265     1  
...imported data.
DEV266
   condition_index condition_label  \
0                0       CorrectGo   
1                1     CorrectStop   

                            raw_beta_description           beta subject  wave  
0    spm_spm:beta (0001) - Sn(1) CorrectGo*bf(1)  beta_0001.nii  DEV266     1  
1  spm_spm:beta (0002) - Sn(1) CorrectStop*bf(1)  beta_000

.....verified that all expected betas exist! Processing...retrieving a mask from the first image in the series beta_0001.nii
...using a combined mask of the first image and the passed in masktemp mask created.
loading files. This step may take some time...
converting to nifti and fdata
   condition_index condition_label  \
0                0       CorrectGo   
1                1     CorrectStop   

                            raw_beta_description           beta subject  wave  
0    spm_spm:beta (0001) - Sn(1) CorrectGo*bf(1)  beta_0001.nii  DEV276     1  
1  spm_spm:beta (0002) - Sn(1) CorrectStop*bf(1)  beta_0002.nii  DEV276     1  
...imported data.
DEV277
   condition_index condition_label  \
0                0       CorrectGo   
1                1     CorrectStop   

                            raw_beta_description           beta subject  wave  
0    spm_spm:beta (0001) - Sn(1) CorrectGo*bf(1)  beta_0001.nii  DEV277     1  
1  spm_spm:beta (0002) - Sn(1) CorrectStop*bf(1)  beta_000

.....verified that all expected betas exist! Processing...retrieving a mask from the first image in the series beta_0001.nii
...using a combined mask of the first image and the passed in masktemp mask created.
loading files. This step may take some time...
converting to nifti and fdata
   condition_index condition_label  \
0                0       CorrectGo   
1                1     CorrectStop   

                            raw_beta_description           beta subject  wave  
0    spm_spm:beta (0001) - Sn(1) CorrectGo*bf(1)  beta_0001.nii  DEV293     1  
1  spm_spm:beta (0002) - Sn(1) CorrectStop*bf(1)  beta_0002.nii  DEV293     1  
...imported data.
DEV294
   condition_index condition_label  \
0                0       CorrectGo   
1                1     CorrectStop   

                            raw_beta_description           beta subject  wave  
0    spm_spm:beta (0001) - Sn(1) CorrectGo*bf(1)  beta_0001.nii  DEV294     1  
1  spm_spm:beta (0002) - Sn(1) CorrectStop*bf(1)  beta_000

.....verified that all expected betas exist! Processing...retrieving a mask from the first image in the series beta_0001.nii
...using a combined mask of the first image and the passed in masktemp mask created.
loading files. This step may take some time...
converting to nifti and fdata
   condition_index condition_label  \
0                0       CorrectGo   
1                1     CorrectStop   

                            raw_beta_description           beta subject  wave  
0    spm_spm:beta (0001) - Sn(1) CorrectGo*bf(1)  beta_0001.nii  DEV304     1  
1  spm_spm:beta (0002) - Sn(1) CorrectStop*bf(1)  beta_0002.nii  DEV304     1  
...imported data.
DEV305
   condition_index condition_label  \
0                0       CorrectGo   
1                1     CorrectStop   

                            raw_beta_description           beta subject  wave  
0    spm_spm:beta (0001) - Sn(1) CorrectGo*bf(1)  beta_0001.nii  DEV305     1  
1  spm_spm:beta (0002) - Sn(1) CorrectStop*bf(1)  beta_000